In [1]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData
from SimplePointNet_normales import SimplePointNet, PointNetAutoEncoder, Decoder, Encoder, SimpleTransformer

from autoencoder_dataset import autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoder
from train_funcs_normales import train_autoencoder_dataloader
from test_funcs_normales import test_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'trimesh' # 'mpi-mesh', 'trimesh'
root_dir = '../Data'

dataset = 'COMA'
name = ''

GPU = True
device_idx = 0
torch.cuda.get_device_name(device_idx)

'NVIDIA RTX A3000 Laptop GPU'

In [2]:
args = {}

generative_model = 'autoencoder'
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'


# below are the arguments for the DFAUST run
reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')
downsample_directory = os.path.join(root_dir, dataset,'template', downsample_method)
ds_factors = [4, 4, 4, 4]
step_sizes = [2, 2, 1, 1, 1]
filter_sizes_enc = [[64, 128], 64, [128, 64, 64]]
filter_sizes_dec = [256]
dilation_flag = True
if dilation_flag:
    dilation=[2, 2, 1, 1, 1] 
else:
    dilation = None
reference_points = [[3567,4051,4597]] #[[414]]  # [[3567,4051,4597]] used for COMA with 3 disconnected components

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed_identity_pointnet_normales',name),
        'results_folder':  os.path.join(root_dir, dataset,'results_identity/pointnet_normales_'+ generative_model),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size':16, 'num_epochs':50, 'eval_frequency':200, 'num_workers': 4,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz':128, 
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,
        'lr':1e-3, 
        'regularization': 5e-5,
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': False,
        'mode':'train', 'shuffle': True, 'nVal': 100, 'normalization': True}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

In [3]:
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    train_vert = np.load(args['data']+'/train.npy')[:,:,0:3]
    test_vert = np.load(args['data']+'/test.npy')[:,:,0:3]
    
    np.save(os.path.join(args['data'],'train_vert.npy'),train_vert)
    del train_vert
    np.save(os.path.join(args['data'],'test_vert.npy'),test_vert)
    del test_vert
    
    shapedata_vert = ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train_vert.npy', 
                          test_file=args['data']+'/test_vert.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    
    mean_vert = shapedata_vert.mean
    std_vert = shapedata_vert.std
    del shapedata_vert
    
    train_norm = np.load(args['data']+'/train.npy')[:,:,3:]
    np.save(os.path.join(args['data'],'train_norm.npy'),train_norm)
    del train_norm
    
    test_norm = np.load(args['data']+'/test.npy')[:,:,3:]
    np.save(os.path.join(args['data'],'test_norm.npy'),test_norm)
    del test_norm
    
    shapedata_norm = ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train_norm.npy', 
                          test_file=args['data']+'/test_norm.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)

    mean_norm = shapedata_norm.mean
    std_norm = shapedata_norm.std
    del shapedata_norm
    
    np.save(args['data']+'/mean.npy', np.hstack((mean_vert, mean_norm)))
    np.save(args['data']+'/std.npy', np.hstack((std_vert, std_norm)))

In [4]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]
    
print("... Done")    

Loading data .. 
... Done


In [5]:
torch.manual_seed(args['seed'])

if GPU:
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

#tspirals = [torch.from_numpy(s).long().to(device) for s in spirals_np]
#tD = [torch.from_numpy(s).float().to(device) for s in bD]
#tU = [torch.from_numpy(s).float().to(device) for s in bU]

cuda:0


In [6]:
# Building model, optimizer, and loss function

dataset_train = autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                           shapedata = shapedata,
                                           normalization = args['normalization'])

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'], num_workers = args['num_workers'])

dataset_val = autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                         shapedata = shapedata,
                                         normalization = args['normalization'])

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])

dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'])



if 'autoencoder' in args['generative_model']:
        model = PointNetAutoEncoder(latent_size=args['nz'],
                                    filter_enc = args['filter_sizes_enc'], 
                                    filter_dec = args['filter_sizes_dec'],  
                                    num_points=shapedata.n_vertex+1, 
                                    device=device).to(device)                  

        
optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

64


In [7]:
if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1

In [8]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 4170436
PointNetAutoEncoder(
  (encoder): Encoder(
    (ptnet): SimplePointNet(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(6, 64, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Sequential(
          (0): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (2): Sequential(
          (0): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (transformers): ModuleList(
        (0): SimpleTransformer(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(6, 64, kernel_size=(1,), stride=(1,))
           

In [9]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    if args['generative_model'] == 'autoencoder':
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn,
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=False,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 34.43it/s]


epoch 0 | tr 0.41489711626744996 | val 0.2928157699108124


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 36.75it/s]


epoch 1 | tr 0.27181661412224645 | val 0.28481658339500426


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 36.73it/s]


...

epoch 47 | tr 0.12987321973062899 | val 0.156399507522583


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 34.57it/s]


epoch 48 | tr 0.1306935332043959 | val 0.1500054430961609


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 32.97it/s]

epoch 49 | tr 0.1296837982111792 | val 0.1586254930496216
~FIN~


In [10]:
args['mode']='test'
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    target, predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, mm_constant = 1000)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)
    np.save(os.path.join(prediction_path,'targets'), target)
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file ../Data/COMA/results_identity/pointnet_normales_autoencoder/latent_128/checkpoints/checkpoint.pth.tar


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 61.36it/s]


autoencoder: normalized loss 0.6568272113800049
autoencoder: euclidean distance in mm= 3.7519407272338867
